In [41]:
import pandas as pd
import os

In [45]:
def load_lexicon(path="librispeech-lexicon.txt"):
    with open(path, 'r') as f:
        lexicon_raw = f.read()
        rows = lexicon_raw.splitlines()
    clean_rows = [row.split() for row in rows]
    lexicon_dict_l = dict()
    for row in clean_rows:
        c_row = row.copy()
        key = c_row.pop(0)
        if len(c_row) == 1:
            c_row[0] = c_row[0] + '_S'
        if len(c_row) >= 2:
            c_row[0] = c_row[0] + '_B'
            c_row[-1] = c_row[-1] + '_E'
        if len(c_row) > 2:
            for i in range(1,len(c_row)-1):
                c_row[i] = c_row[i] + '_I'
        val = " ".join(c_row)
        lexicon_dict_l[key] = val
    return lexicon_dict_l

In [44]:
user_id = "0000"
utterance =  "THE PAPER PUBLISHED NO DETAILS AND THERE WAS NO ATTACK"
wav_path = "/working/egs/gop_speechocean762/s5/data/speechocean762/WAVE/SPEAKER1030/010300105.WAV"

sample = {
    "user_id":[user_id, ],
    "utterance":[utterance, ],
    "wav_path":[wav_path, ]
}
dataset = pd.DataFrame(sample)
# dataset.word = dataset.word.apply(lambda x: x.split())
# dataset = dataset.explode(column="word")
dataset.head()

,user_id,utterance,wav_path
0,0000,THE PAPER PUBLISHED NO DETAILS AND THERE WAS N...,/working/egs/gop_speechocean762/s5/data/speech...


In [47]:
lexicon_dict_l = load_lexicon("/home/tuyendv/projects/pronunciation_scoring/librispeech-lexicon.txt")

In [53]:
base_path = "test"
scp_file = f'{base_path}/wav.scp'
spk2utt = f'{base_path}/spk2utt'
utt2spk = f'{base_path}/utt2spk'
text = f'{base_path}/text'
text_phone =  f'data/local/text-phone'

scp_str = ""
spk2utt_str = ""
utt2spk_str = ""
text_str = ""
text_phone_str = ""

for index in dataset.index:
    wav_id = os.path.basename(dataset["wav_path"][index]).rstrip(".WAV")
    
    _scp_str = f'{wav_id}\t{dataset["wav_path"][index]}\n'
    _text_str = f'{wav_id}\t{dataset["utterance"][index]}\n'
    _utt2spk_str = f'{wav_id}\t{dataset["user_id"][index]}\n'
    _spk2utt_str = f'{dataset["user_id"][index]}\t{wav_id}\n'
        
    for i, word in enumerate(dataset["utterance"][index].split()):
        _text_phone_str = f'{wav_id}.{i}\t{lexicon_dict_l[word]}\n'
        text_phone_str += _text_phone_str
        
    scp_str += _scp_str
    text_str += _text_str
    utt2spk_str += _utt2spk_str
    spk2utt_str += _spk2utt_str
        

with open(scp_file, 'w') as f:
        f.write(scp_str)

with open(spk2utt, 'w') as f:
    f.write(spk2utt_str)

with open(utt2spk, 'w') as f:
    f.write(utt2spk_str)

with open(text, 'w') as f:
    f.write(text_str)

with open(text_phone, 'w') as f:
    f.write(text_phone_str)

In [ ]:


scp_str = ""
spk2utt_str = ""
utt2spk_str = ""
text_str = ""
text_phone = ""


with open(scp_file, 'w') as f:
    f.write(scp_str)

with open(spk2utt, 'w') as f:
    f.write(spk2utt_str)

with open(utt2spk, 'w') as f:
    f.write(utt2spk_str)

with open(text, 'w') as f:
    f.write(text_str)

with open("text-phone", 'w') as f:
    f.write(text_phone)

In [36]:
scp_file = "wav.scp"
spk2utt = "spk2utt"
utt2spk = "utt2spk"
text = "text"
scp_str = ""
spk2utt_str = ""
utt2spk_str = ""
text_str = ""
text_phone = ""
id_list = ["%04d" % x for x in range(10000)]
speaker_hist = set()
dataset.sort_values(by="id_user", ascending=True, inplace=True)
j = 0
holder = ""
cache = ""
for _, line in dataset.iterrows():
    speaker_hist.add(line.id_user)
    spk = id_list[len(speaker_hist)]
    if cache == "":
        cache = spk
    utt = spk + id_list[j]
    j += 1
    scp_str = scp_str + utt + ' ' + f"WAVE/{line.file_name}\n"
    if cache != spk:
        spk2utt_str = spk2utt_str + holder + '\n'
        holder = ""
    if len(holder)==0:
        holder = spk + " " + utt
    else:
        holder = holder + " " + utt
    cache = spk
    utt2spk_str = utt2spk_str + utt + " " + spk + '\n'
    text_str = text_str + utt + " " + line.word.upper() + '\n' 
    print(line.word)
    text_phone = text_phone + utt + ".0\t" + lexicon_dict_l[line.word.upper()] + '\n'
else:
    spk2utt_str = spk2utt_str + holder + '\n'

with open(scp_file, 'w') as f:
    f.write(scp_str)

with open(spk2utt, 'w') as f:
    f.write(spk2utt_str)

with open(utt2spk, 'w') as f:
    f.write(utt2spk_str)

with open(text, 'w') as f:
    f.write(text_str)

with open("text-phone", 'w') as f:
    f.write(text_phone)

THE
PAPER
PUBLISHED
NO
DETAILS
AND
THERE
WAS
NO
ATTACK
